In [ ]:
#install dependency
!pip install protein-bert
!pip install gdown
!pip install biopython

In [ ]:
#download assets
!gdown --folder '1MMvRjjeR1Hz6hw08WZhZwLI-89pUpYtc'

In [6]:
#import packages
import os
import tensorflow as tf
from Bio import SeqIO
import os
import pandas as pd
import os
import pandas as pd
from IPython.display import display
from tensorflow import keras
from sklearn.model_selection import train_test_split
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
import numpy as np
from proteinbert.finetuning import encode_categorical_Y
from proteinbert.finetuning import encode_Y
from proteinbert.finetuning import encode_dataset
from io import StringIO

In [ ]:
#load the predictors
def get_models(size = 1024):
    df = pd.read_csv('GT_MODELS/label_to_family.csv',index_col=[0])
    unique_labels = sorted(set(df['label'].unique()))
    label_dict = dict(zip(df['label'],df['FamilyId']))
    OUTPUT_TYPE = OutputType(False, 'categorical')
    OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, unique_labels)

    pretrained_model_generator, input_encoder = load_pretrained_model(
    local_model_dump_dir='',
    validate_downloading=False)

    # get_model_with_hidden_layers_as_outputs gives the model output access to the hidden layers (on top of the output)
    model_generator = FinetuningModelGenerator(
        pretrained_model_generator, 
        OUTPUT_SPEC, 
        pretraining_model_manipulation_function = get_model_with_hidden_layers_as_outputs,
        dropout_rate = 0.5)


    fold=0
    fine_tuned_model_0 = model_generator.create_model(size)
    fine_tuned_model_0.load_weights('GT_MODELS/fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))

    fold=1
    fine_tuned_model_1 = model_generator.create_model(size)
    fine_tuned_model_1.load_weights('GT_MODELS/fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))

    fold=2
    fine_tuned_model_2 = model_generator.create_model(size)
    fine_tuned_model_2.load_weights('GT_MODELS/fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))

    fold=3
    fine_tuned_model_3 = model_generator.create_model(size)
    fine_tuned_model_3.load_weights('GT_MODELS/fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))

    fold=4
    fine_tuned_model_4 = model_generator.create_model(size)
    fine_tuned_model_4.load_weights('GT_MODELS/fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))    

    models = [
        fine_tuned_model_0,
        fine_tuned_model_1,
        fine_tuned_model_2,
        fine_tuned_model_3,
        fine_tuned_model_4,
    ]
    return models, input_encoder, OUTPUT_SPEC, label_dict

models, input_encoder, OUTPUT_SPEC, label_dict = get_models()

In [ ]:
fasta_entry='''
>tr|A0A084A710|A0A084A710_LACLC Putative Glycosiltransferase OS=Lactococcus lactis subsp. cremoris GE214 OX=1415168 GN=U725_02784 PE=4 SV=1
MLYDRGINVCHTTIYRWVQEYSKVLYYLWKKKNRQSFYSWKMDETYIKIKGRWHYLYRAI
DADGLTLDIWLRKKRETQAAYAFLKRLHKQFGEPKAIVTDKAPSLGSAFRKLQCWSFECT
DLMLPFAYKCYNIRLQYCIFFTEVLPMTVRKRQFGTEKDSFALPVRHTRENDRKLSINRI
FIVITLIFWVCYVSSTIYNQMIVHIPNFRSAIETCLYIVIVTLLDFSALMYLIEREGALK
RFAQHTRVPRQNIDTHFSKRQPKMTVLVPSYDEEIKVIRQTLLSAALQEYPDINIVLLLD
DKPRPSQQDAIDKLERTKAAVLDIGKRFEEPSHRLKQSLLAFENKTKHKKQSKTDDAFEA
AQKVVQEYKWAADWLMALANSEKVADHVDHFFVDKVIKDLADNLYLISSAIDKSISENVI
ISDERLHQLYSRLVWVFTFNISYFQRKEYLSLSHEANKAMNLNSYINLMGGKYKIEKIAK
NKVLLPVQEHETFDLEVPDSEYILTLDADSILLMEYCLRLVYLMDQKQNEKIAVTQTPYS
SFRGSPTRIERITGSTTDIQHLLHQGSTHFDATFWVGANAVIRKRALEDIAEVDEIHGFM
FRRFIQDRTVIEDTESSIDLRRKEWRLYNYPERLSYSATPPDFGSLVIQRGRWANGGLLI
LPKLLAARKQLKREGRPMRRLEFLIRMNYMCSVAWASFGLLFLLVYPFDSKLLSIFILLS
AIPYFIAMSSDLKYLRYRRIDVFGIYGFNLLLLPVNLAGVMKSIQQGLTDKKIPFARTPK
VDNRTSVRGWYVFSPIAIIVYSMLTCLKSYQDGNWANALFAGFNTVTALWATIAYIGLMN
MVKDILNNFLNLILVEEKSKKIENAIEQSDEMDWRSILYYGEKEEKLSLRVAIEHHKEH
>tr|A0A7L8YR83|A0A7L8YR83_9CAUD Bactoprenol glycosiltransferase OS=Escherichia phage HF4s OX=2775262 GN=HF4s_0017 PE=4 SV=1
MRNEDISGRSCLQRRGSDTYFYKTVREFEELKPYEVEIVFINDGSKDATESIINALSVSD
PLVVPLSFTRNFGKEPALFAGLDHATGDAVIPIDVDLQDPIEVIPHLIEKWLAGADMVLA
KRSDRSTDGRLKRKTAEWFYKLHNKISNPQIEENVGDFRLMSREVVENIKLMPERNLFMK
GVLSWVGGRTDVVEYARAERVAGSTKFNGWKLWNLALEGITSFSTFPLRMWTYIGLFVAG
AAFMYGAWMIIDTLAFGNAVRGYPSLLVSILFLGGIQLIGIGVLGEYIGRIYVEVKNRPR
YVLKGRK
>tr|Q9XC63|Q9XC63_PSEAI Glycosiltransferase OS=Pseudomonas aeruginosa OX=287 GN=wbjA PE=4 SV=1
MLLSVVVSTWRRPKELKLVLAGLVAQSLAFKDFEVIVVDSNSGDETPDVVREFEAVQGFN
VRLEQAELNSASAKRNKGVRLAKGRYVVLLDDDCVPDPAHLSNFLNAANSRRGERVIWCG
GVRFSCSLVSTSNYYRYRDSCHFSADKPLPNSLGYKTIVTMNMLAERDILLQDNALFDER
FVGYGFEDTEFGLRVGKLGYSVLPCRAEIEHVEPEGSISKFCRKYYHAARDGMPEFMKVA
TTDDLETLGPIRWMEPGFALSSKFQSLFRAVLCLALESKFSNILIHSLEKTDSIRWLYSR
AAYRLVLASSYRKGVLDRKSKLSKEKANKDGWYS
>tr|A9H7V6|A9H7V6_GLUDA Glycosyl transferase group 1 OS=Gluconacetobacter diazotrophicus (strain ATCC 49037 / DSM 5601 / CCUG 37298 / CIP 103539 / LMG 7603 / PAl5) OX=272568 GN=GDI3747 PE=4 SV=1
MSLSIGIVTHSTNPRGGVVHGMALAEALCDAGHDATLIAPDATGAGFFRAPRCAVRCIPV
RPERDLPVQVERRIGELREALRGAPHDVLHAQDPIGANALADLVREGGLRGFARTVHHLD
HFSHPGLAARQARGVTAATALFTVSTLWQGILADEYGRMAPVVGNGVDAVRFSPEPGPCD
GTLRARYRLPPEGGLVLSVGGIERRKNTLMLLDAFLALRRERPDLHLVVAGGASLLDHSE
YRRLFDRRVRDSGCAGAVTVTGPVADGDMPSFYRQALVLAYPSRTEGFGLCPLEALACGT
PVIVPDAAPFTEHLHGPEVLWCAPDRPETLVAALRDALGARARFRAGGPATARRFDWRGV
AARHMAAYRRLAALPHVTTPPGPKREYRDHA
'''


In [8]:
#create the inputs
#a list of protein sequences 
#and a list of zeros

fasta_io = StringIO(fasta_entry)
records = SeqIO.parse(fasta_io, "fasta")

ids = []
seqs = []
for n in records:
    ids.append(n.id)
    seqs.append(str(n.seq))
    if len(str(n.seq)) > 1000:
        raise TypeError("Sequence must be less than 1000AA")

targets = [0 for n in seqs]
print('predicting:',len(ids),len(seqs),len(targets),'proteins')


In [10]:
#encode the inputs
X, y_true, sample_weights = encode_dataset(seqs, targets, input_encoder, OUTPUT_SPEC, \
                seq_len = 512*2, needs_filtering = False)

In [13]:
#prepare the result dataframe
in_df = pd.DataFrame()
in_df['ids']=ids
in_df['seq']=seqs
in_df.head()

In [ ]:
#predict
from tqdm.auto import tqdm
for index, model in tqdm(enumerate(models)):
    preds = model.predict(X, batch_size = 32)
    index_sorted = np.argsort(preds, axis=1)
    
    labels_1 = pd.Series(index_sorted[:,-1])
    labels_2 = pd.Series(index_sorted[:,-2])
    
    family_1 = labels_1.map(label_dict)
    family_2 = labels_2.map(label_dict)
    
    prob_1 = preds[np.arange(len(X[0])), labels_1]
    prob_2 = preds[np.arange(len(X[0])), labels_2]
    
    in_df['model_'+str(index)+'_labels_1']=labels_1.values
    in_df['model_'+str(index)+'_family_1']=family_1.values
    in_df['model_'+str(index)+'_prob_1']=prob_1


In [16]:
#parse outputs and get the majority prediction/probability
def select(X):
    temp = []
    items = list(set(X))
    to_count = list(X)
    #print(temp,items,to_count)
    for n in items:
        temp.append((to_count.count(n),n))
    temp.sort()
    #print(temp)
    return temp[-1][-1]


def vote_mean(X):
    #print(X)
    majority = X['vote']
    score = []
    for a,b in zip(X[['model_0_family_1','model_1_family_1','model_2_family_1','model_3_family_1','model_4_family_1']],
                   X[['model_0_prob_1','model_1_prob_1','model_2_prob_1','model_3_prob_1','model_4_prob_1']]):
        if a==majority:
            score.append(b)
    return np.mean(score)
                   


def get_dict_from_desc(X):
    temp_dict = {}
    item_list= X.split('|')
    #print(item_list)
    for item in item_list:
        if '=' in item:
            key = item.split('=')[0].strip()
            value = item.split('=')[1].strip()
            temp_dict[key]=value
            #print(key,value)
    return temp_dict

def get_desc(X, target):
    temp_dict = get_dict_from_desc(X)
    #print(temp_dict)
    return temp_dict[target]


In [ ]:
in_df['vote']=in_df[['model_0_family_1','model_1_family_1','model_2_family_1','model_3_family_1','model_4_family_1']].apply(select,axis=1)
in_df['vote_mean']=in_df.apply(vote_mean,axis=1)

In [ ]:
#visualize result
in_df[['ids','vote','vote_mean']]